In [ ]:
import mssql_python
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

print("✅ Libraries imported successfully!")
print(f"   mssql_python version: {mssql_python.__version__}")

## 🔐 Step 1: Configure Credentials

Update these with your Azure credentials:

In [ ]:
# ============================================================
# 🔧 CONFIGURATION - Azure SQL + Azure OpenAI
# ============================================================

# Azure SQL Database connection string (Active Directory Interactive)
CONNECTION_STRING = (
    "Server=azure-ug-demo.database.windows.net;"
    "Database=azure-ug-demo;"
    "Authentication=ActiveDirectoryInteractive;"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

# Azure OpenAI credentials (loaded securely from .env file)
AZURE_OPENAI_ENDPOINT = "https://azureugdemo-resource.cognitiveservices.azure.com/"
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_DEPLOYMENT = "gpt-5.2-chat"  # New model deployment

# Verify credentials loaded
if AZURE_OPENAI_KEY:
    print("✅ Credentials configured!")
    print("   Server: azure-ug-demo.database.windows.net")
    print("   Database: azure-ug-demo")
    print("   Auth: ActiveDirectoryInteractive")
    print("   OpenAI Endpoint: azureugdemo-resource.cognitiveservices.azure.com")
    print(f"   Model: {AZURE_OPENAI_DEPLOYMENT}")
    print("   API Key: ✓ Loaded from .env file")
else:
    print("⚠️  API Key not found! Create a .env file with:")
    print('   AZURE_OPENAI_KEY=your-api-key')

## 🔌 Step 2: Connect to Database

Using **mssql-python** for fast, native connectivity:

In [ ]:
# Connect to Azure SQL Database
connection = mssql_python.connect(CONNECTION_STRING)

print("✅ Connected to Azure SQL Database!")
print("   Using: mssql-python (fast native driver)")

## 🧠 Step 3: Connect to Azure OpenAI

In [ ]:
# Initialize Azure OpenAI client
ai_client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version="2025-04-01-preview"
)

print("✅ Connected to Azure OpenAI!")
print(f"   Model: {AZURE_OPENAI_DEPLOYMENT}")
print(f"   API Version: 2025-04-01-preview")

---

# 🎬 THE MAGIC BEGINS!

Now let's see the 5-step flow in action:

```
Question → Schema → SQL → Results → Answer
```

---

## 📋 Step 4: Fetch Database Schema

This is the **secret sauce** - we tell the AI what tables and columns exist!

In [ ]:
def get_database_schema(conn):
    """
    Fetch schema information from the database.
    Returns a formatted string describing all tables and columns.
    """
    cursor = conn.cursor()
    
    # Query INFORMATION_SCHEMA to get all tables and columns
    schema_query = """
    SELECT 
        t.TABLE_NAME,
        c.COLUMN_NAME,
        c.DATA_TYPE
    FROM INFORMATION_SCHEMA.TABLES t
    JOIN INFORMATION_SCHEMA.COLUMNS c 
        ON t.TABLE_NAME = c.TABLE_NAME
    WHERE t.TABLE_TYPE = 'BASE TABLE'
    ORDER BY t.TABLE_NAME, c.ORDINAL_POSITION
    """
    
    cursor.execute(schema_query)
    rows = cursor.fetchall()
    
    # Build schema description
    schema_dict = {}
    for row in rows:
        table_name, column_name, data_type = row
        if table_name not in schema_dict:
            schema_dict[table_name] = []
        schema_dict[table_name].append(f"{column_name} ({data_type})")
    
    # Format as readable string
    schema_text = "DATABASE SCHEMA:\n"
    for table, columns in schema_dict.items():
        schema_text += f"\nTable: {table}\n"
        schema_text += f"  Columns: {', '.join(columns)}\n"
    
    cursor.close()
    return schema_text, len(schema_dict)

# Fetch the schema
schema, table_count = get_database_schema(connection)

print(f"✅ Found {table_count} tables in the database!")
print("\n" + "="*60)
print(schema)
print("="*60)

## 🤖 Step 5: Define the AI SQL Generator

This function sends the question + schema to Azure OpenAI and gets back SQL:

In [ ]:
def generate_sql(question, schema):
    """
    Use Azure OpenAI to convert natural language to SQL.
    The schema context helps AI generate accurate queries!
    """
    
    prompt = f"""You are a SQL expert. Given the database schema and a question, 
generate a SQL query to answer it.

{schema}

Rules:
- Return ONLY the SQL query, no explanations
- Use proper SQL Server syntax
- Only query tables that exist in the schema above

Question: {question}

SQL Query:"""

    response = ai_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You are a SQL query generator. Return only valid SQL."},
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=500
    )
    
    sql = response.choices[0].message.content.strip()
    # Clean up markdown formatting if present
    sql = sql.replace("```sql", "").replace("```", "").strip()
    return sql

print("✅ SQL Generator function defined!")

## ⚡ Step 6: Define the Query Executor

This runs the SQL using **mssql-python** (the fast part!):

In [ ]:
def execute_query(conn, sql):
    """
    Execute SQL query using mssql-python and return results.
    """
    cursor = conn.cursor()
    cursor.execute(sql)
    
    # Get column names from cursor description
    columns = [desc[0] for desc in cursor.description]
    
    # Fetch all rows
    rows = cursor.fetchall()
    cursor.close()
    
    return columns, rows

print("✅ Query Executor function defined!")

## 💬 Step 7: Define the Response Formatter

This asks AI to explain the results in plain English:

In [ ]:
def format_response(question, columns, rows):
    """
    Use Azure OpenAI to create a friendly, conversational response.
    """
    
    # Format data as text (limit to 20 rows for context size)
    data_text = f"Columns: {columns}\nData: {rows[:20]}"
    
    prompt = f"""Based on this database query result, answer the user's question 
in a friendly, conversational way.

Question: {question}
Query Results: {data_text}

Provide a helpful summary:"""

    response = ai_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that explains data in simple terms."},
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=500
    )
    
    return response.choices[0].message.content

print("✅ Response Formatter function defined!")

---

# 🎯 THE COMPLETE CHATBOT

Putting it all together in one function:

---

In [ ]:
def chat(question):
    """
    Main chatbot function - orchestrates the entire flow:
    Question → Schema → SQL → Results → Answer
    """
    
    # Display question as formatted header
    display(Markdown(f"## 📝 Question\n> {question}"))
    
    # Step 1: Get schema (already fetched, reuse it)
    display(Markdown(f"**🔍 Step 1:** Using database schema... ✓ *{table_count} tables available*"))
    
    # Step 2: Generate SQL
    display(Markdown("**🤖 Step 2:** Generating SQL query..."))
    sql = generate_sql(question, schema)
    display(Markdown(f"```sql\n{sql}\n```"))
    
    # Step 3: Execute query
    display(Markdown("**⚡ Step 3:** Executing query with mssql-python..."))
    columns, rows = execute_query(connection, sql)
    display(Markdown(f"✓ *Found {len(rows)} results*"))
    
    # Step 4: Format response
    display(Markdown("**💬 Step 4:** Formatting response..."))
    response = format_response(question, columns, rows)
    
    # Show the answer with nice formatting
    display(Markdown("---"))
    display(Markdown("## 🎯 Answer"))
    display(Markdown(response))
    display(Markdown("---"))
    
    return response

print("✅ Chatbot is ready! Use chat('your question') to try it.")

---

# 🚀 TRY IT OUT!

Ask questions in natural language:

---

In [ ]:
# 🎬 DEMO TIME! Try these questions:

chat("Who are my top 5 customers by total orders?")

In [ ]:
# Try another question!

chat("What products are running low in stock?")

In [ ]:
# One more!

chat("Show me sales trends for the last months")

In [ ]:
# 🎤 YOUR TURN! Ask any question about your data:

chat("What is the average order value for each customer segment?")

---

## 🧹 Cleanup

Close the database connection when done:

In [ ]:
# Close connection when done
connection.close()
print("✅ Connection closed!")

---

# 📚 Key Takeaways

| Component | What It Does | Why It's Great |
|-----------|--------------|----------------|
| **mssql-python** | Database connectivity | ⚡ Fast native driver |
| **Azure OpenAI** | Natural language → SQL | 🧠 Smart query generation |
| **Schema Context** | Tells AI about tables | 🎯 Accurate queries |

---

## 🔗 Resources

- [mssql-python GitHub](https://github.com/microsoft/mssql-python)
- [Azure OpenAI Docs](https://learn.microsoft.com/azure/ai-services/openai/)
- [Azure SQL Database](https://learn.microsoft.com/azure/azure-sql/)

---

*Demo created for Azure User Group - December 2025* 🎉